In [ ]:
# prompt: import pyspark library

!pip install pyspark


In [ ]:
!pip install delta-spark

**Import libraries**

In [ ]:
import pyspark
from pyspark.sql import SparkSession

**Mount google drive**

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from google.colab import drive
import os

# Now mount the drive
drive.mount('/content/drive/')

# Specify the datasets_interview folder path
datasets_interview_path = '/content/drive/MyDrive/datasets_interview'  # Replace with your actual path

# Check if the directory exists
if os.path.exists(datasets_interview_path):
    # List files in the datasets_interview directory
    for filename in os.listdir(datasets_interview_path):
        print(filename)
else:
    print(f"Error: Directory '{datasets_interview_path}' not found.")

**Load dataset**

In [ ]:
spark = SparkSession.builder.getOrCreate()
file_path_google="/content/drive/MyDrive/datasets_interview/google_dataset.csv"
df_google=spark.read.csv(file_path_google, header=True, inferSchema=True)

In [ ]:
df_google.show(100)

**Show all acountires for google dataset**

In [ ]:
from pyspark.sql.functions import col

# Assuming 'country' is the column name containing country information
# Filter the DataFrame to include only rows where the 'country' column is not null
filtered_df = df_google.filter(col("country_name").isNotNull())

# Iterate through the filtered DataFrame and extract all unique countries
distinct_countries = set()
for row in filtered_df.collect():
    country = row["country_name"]
    distinct_countries.add(country)


# Print the distinct countries
for country in distinct_countries:
  print(country)

In [ ]:
print(len(distinct_countries))

**Clean dataset for country romania**

In [ ]:
df_google.count()

In [ ]:
df_google.printSchema()

In [ ]:
df_google_cleaned=df_google.fillna("0")

In [ ]:
df_google_cleaned.createOrReplaceTempView("csv_table")

In [ ]:
# Assuming df_google is your DataFrame
df_google_deduplicated = df_google.dropDuplicates()

# Show the deduplicated DataFrame
df_google_deduplicated.count()

In [ ]:
# prompt: fill null values with 0 from df_google_dedubliplicat

df_google_removenulls = df_google_deduplicated.fillna(0)
df_google_removenulls.show(1000)


**Clean data that countains invalid country codes**

In [ ]:
from pyspark.sql.functions import when

# Define a mapping dictionary for country codes
country_code_mapping = {
    "ghana": "gh", "hungary": "hu", "finland": "fi", "st kitts and nevis": "kn",
    "georgia": "ge", "guatemala": "gt", "latvia": "lv", "peru": "pe", "Åland": "ax",
    "germany": "de", "french polynesia": "pf", "italy": "it", "zambia": "zm",
    "liberia": "lr", "solomon islands": "sb", "samoa": "ws", "portugal": "pt",
    "netherlands": "nl", "nicaragua": "ni", "bermuda": "bm", "maldives": "mv",
    "u.s. virgin islands": "vi", "barbados": "bb", "brazil": "br", "mozambique": "mz",
    "kyrgyzstan": "kg", "senegal": "sn", "kenya": "ke", "faroe islands": "fo",
    "mauritania": "mr", "new caledonia": "nc", "united arab emirates": "ae",
    "south africa": "za", "indonesia": "id", "puerto rico": "pr", "albania": "al",
    "chile": "cl", "uruguay": "uy", "saint vincent and the grenadines": "vc",
    "guinea": "gn", "montenegro": "me", "russia": "ru", "cameroon": "cm", "iran": "ir",
    "slovenia": "si", "yemen": "ye", "cuba": "cu", "armenia": "am", "zimbabwe": "zw",
    "malaysia": "my", "spain": "es", "bulgaria": "bg", "philippines": "ph",
    "belarus": "by", "canada": "ca", "united kingdom": "gb", "sri lanka": "lk",
    "lebanon": "lb", "iceland": "is", "el salvador": "sv", "belgium": "be",
    "egypt": "eg", "iraq": "iq", "saudi arabia": "sa", "botswana": "bw",
    "singapore": "sg", "benin": "bj", "bolivia": "bo", "bosnia and herzegovina": "ba",
    "vietnam": "vn", "algeria": "dz", "burundi": "bi", "nigeria": "ng",
    "afghanistan": "af", "south korea": "kr", "angola": "ao", "mexico": "mx",
    "liechtenstein": "li", "austria": "at", "qatar": "qa", "bahrain": "bh",
    "israel": "il", "gabon": "ga", "uganda": "ug", "andorra": "ad", "hong kong": "hk",
    "bahamas": "bs", "thailand": "th", "greenland": "gl", "seychelles": "sc",
    "bangladesh": "bd", "aruba": "aw", "fiji": "fj", "myanmar": "mm", "venezuela": "ve",
    "bhutan": "bt", "haiti": "ht", "togo": "tg", "estonia": "ee",
    "hashemite kingdom of jordan": "jo", "australia": "au", "lesotho": "ls",
    "mali": "ml", "japan": "jp", "denmark": "dk", "new zealand": "nz",
    "luxembourg": "lu", "papua new guinea": "pg", "suriname": "sr",
    "dominican republic": "do", "niger": "ne", "tajikistan": "tj", "paraguay": "py",
    "burkina faso": "bf", "laos": "la", "ireland": "ie", "saint lucia": "lc",
    "jamaica": "jm", "republic of lithuania": "lt", "north macedonia": "mk",
    "croatia": "hr", "ukraine": "ua", "guinea-bissau": "gw", "costa rica": "cr",
    "argentina": "ar", "brunei": "bn", "ecuador": "ec", "libya": "ly", "morocco": "ma",
    "sierra leone": "sl", "central african republic": "cf", "colombia": "co",
    "honduras": "hn", "kazakhstan": "kz", "american samoa": "as", "panama": "pa",
    "south sudan": "ss", "cambodia": "kh", "serbia": "rs", "french guiana": "gf",
    "romania": "ro", "nepal": "np", "turkey": "tr", "poland": "pl", "mauritius": "mu",
    "guam": "gu", "czechia": "cz", "india": "in", "slovakia": "sk", "namibia": "na",
    "belize": "bz", "tonga": "to", "rwanda": "rw", "gambia": "gm", "curaçao": "cw",
    "azerbaijan": "az", "france": "fr", "madagascar": "mg", "taiwan": "tw",
    "malta": "mt", "sweden": "se", "trinidad and tobago": "tt", "united states": "us",
    "uzbekistan": "uz", "reunion": "re", "switzerland": "ch", "guadeloupe": "gp",
    "martinique": "mq", "sudan": "sd", "pakistan": "pk", "republic of moldova": "md",
    "kuwait": "kw", "comoros": "km", "malawi": "mw", "greece": "gr", "china": "cn",
    "dominica": "dm", "guyana": "gy", "isle of man": "im", "oman": "om",
    "sint maarten": "sx", "norway": "no", "somalia": "so", "jersey": "je",
    "vanuatu": "vu", "ethiopia": "et", "tunisia": "tn"
}

In [ ]:
from pyspark.sql.functions import when, regexp_replace, col, lit
def add_country_code(df, country_code_mapping):
    country_code_expr = None
    for country, code in country_code_mapping.items():
        if country_code_expr is None:
            country_code_expr = when(col("country_name") == country, lit(code))
        else:
            country_code_expr = country_code_expr.when(col("country_name") == country, lit(code))

    # Add the new column
    return df.withColumn("country_code", country_code_expr)

In [ ]:
# Update the DataFrame with the new country_code column
df_google = add_country_code(df_google, country_code_mapping)

# Show updated DataFrame
df_google.show(1000)

In [ ]:
# Select distinct country_name and country_code pairs
distinct_countries = df_google.select("country_name", "country_code").distinct()
n=distinct_countries.count()
# Show the distinct pairs of country names and country codes
distinct_countries.show(n,truncate=False)

In [ ]:
print(n)

In [ ]:
df_google.count()

**Delete all rows that contain null addresses**

In [ ]:
from pyspark.sql.functions import col

# Filter out rows where the 'address' column is null
df_google_Filter_out = df_google.filter(col("address").isNotNull())

# Show the cleaned DataFrame
df_google_Filter_out.count()

In [ ]:
df_google_Filter_out.show()

**Clean data that contain invalid prefix phone country**

In [ ]:
# Dictionary mapping country names to their international dialing prefixes
prefix_mapping = {
    "brunei": "673",
    "india": "91",
    "sweden": "46",
    "vanuatu": "678",
    "nepal": "977",
    "bangladesh": "880",
    "gabon": "241",
    "kazakhstan": "7",
    "israel": "972",
    "romania": "40",
    "somalia": "252",
    "united states": "1",
    "el salvador": "503",
    "saint lucia": "1758",
    "austria": "43",
    "niger": "227",
    "russia": "7",
    "hong kong": "852",
    "suriname": "597",
    "bulgaria": "359",
    "qatar": "974",
    "panama": "507",
    "latvia": "371",
    "mauritius": "230",
    "guam": "1671",
    "venezuela": "58",
    "japan": "81",
    "serbia": "381",
    "greece": "30",
    "italy": "39",
    "denmark": "45",
    "comoros": "269",
    "netherlands": "31",
    "china": "86",
    "papua new guinea": "675",
    "bahrain": "973",
    "ireland": "353",
    "tonga": "676",
    "zambia": "260",
    "fiji": "679",
    "dominica": "1767",
    "faroe islands": "298",
    "czechia": "420",
    "liberia": "231",
    "switzerland": "41",
    "guinea": "224",
    "republic of lithuania": "370",
    "egypt": "20",
    "senegal": "221",
    "ukraine": "380",
    "armenia": "374",
    "philippines": "63",
    "zimbabwe": "263",
    "benin": "229",
    "aruba": "297",
    "sudan": "249",
    "haiti": "509",
    "bahamas": "1242",
    "republic of moldova": "373",
    "saudi arabia": "966",
    "chile": "56",
    "iceland": "354",
    "kyrgyzstan": "996",
    "guyana": "592",
    "south africa": "27",
    "trinidad and tobago": "1868",
    "mexico": "52",
    "united kingdom": "44",
    "uzbekistan": "998",
    "sierra leone": "232",
    "cuba": "53",
    "tunisia": "216",
    "slovakia": "421",
    "georgia": "995",
    "yemen": "967",
    "dominican republic": "1809",
    "cambodia": "855",
    "argentina": "54",
    "ghana": "233",
    "slovenia": "386",
    "indonesia": "62",
    "myanmar": "95",
    "peru": "51",
    "belgium": "32",
    "tajikistan": "992",
    "gambia": "220",
    "belarus": "375",
    "liechtenstein": "423",
    "estonia": "372",
    "iraq": "964",
    "uruguay": "598",
    "libya": "218",
    "mauritania": "222",
    "singapore": "65",
    "colombia": "57",
    "luxembourg": "352",
    "paraguay": "595",
    "barbados": "1246",
    "poland": "48",
    "spain": "34",
    "burkina faso": "226",
    "mali": "223",
    "portugal": "351",
    "algeria": "213",
    "curaçao": "599",
    "north macedonia": "389",
    "samoa": "685",
    "sint maarten": "1721",
    "thailand": "66",
    "nigeria": "234",
    "martinique": "596",
    "costa rica": "506",
    "namibia": "264",
    "french polynesia": "689",
    "germany": "49",
    "croatia": "385",
    "oman": "968",
    "hungary": "36",
    "united arab emirates": "971",
    "kuwait": "965",
    "canada": "1",
    "uganda": "256",
    "brazil": "55",
    "albania": "355",
    "taiwan": "886",
    "mozambique": "258",
    "seychelles": "248",
    "puerto rico": "1787",
    "malta": "356",
    "u.s. virgin islands": "1340",
    "bosnia and herzegovina": "387",
    "south korea": "82",
    "rwanda": "250",
    "central african republic": "236",
    "andorra": "376",
    "sri lanka": "94",
    "ecuador": "593",
    "morocco": "212",
    "guadeloupe": "590",
    "afghanistan": "93",
    "azerbaijan": "994",
    "guatemala": "502",
    "botswana": "267",
    "togo": "228",
    "malaysia": "60",
    "solomon islands": "677",
    "norway": "47",
    "burundi": "257",
    "montenegro": "382",
    "cameroon": "237",
    "bolivia": "591",
    "angola": "244",
    "malawi": "265",
    "honduras": "504",
    "belize": "501",
    "new zealand": "64",
    "pakistan": "92",
    "vietnam": "84",
    "bhutan": "975",
    "australia": "61",
    "jamaica": "1876",
    "turkey": "90",
    "åland": "358",
    "nicaragua": "505",
    "finland": "358",
    "ethiopia": "251",
    "madagascar": "261",
    "laos": "856",
    "lebanon": "961",
    "maldives": "960",
    "france": "33",
    "saint vincent and the grenadines": "1784",
    "french guiana": "594",
    "reunion": "262",
    "lesotho": "266",
    "american samoa": "1684",
    "kenya": "254",
    "south sudan": "211",
    "guinea-bissau": "245",
    "isle of man": "44",
    "greenland": "299",
    "new caledonia": "687",
    "jersey": "44",
    "hashemite kingdom of jordan": "962",
    "bermuda": "1441",
    "iran": "98",
    "st kitts and nevis": "1869"
}

In [ ]:
from pyspark.sql.functions import lower, col, regexp_replace

def filter_by_prefix(df, prefix_mapping):
    """
    Filters the given DataFrame based on phone number prefixes mapped to country names.

    Parameters:
    df (DataFrame): The input DataFrame containing 'country_name' and 'phone' columns.
    prefix_mapping (dict): Dictionary mapping country names to phone number prefixes.

    Returns:
    DataFrame: A filtered DataFrame containing rows that match the prefix conditions.
    """
    # Build a filter condition by iterating through the prefix mapping.
    conditions = None
    for country, prefix in prefix_mapping.items():
        # Remove +, ( and ) from the phone column before checking the prefix.
        cleaned_phone = regexp_replace(col("phone"), r"[+()]", "")
        cond = (lower(col("country_name")) == country) & cleaned_phone.startswith(prefix)
        conditions = cond if conditions is None else conditions | cond

    # Apply the combined condition to filter the DataFrame
    return df.filter(conditions)

In [ ]:
df_filtered_prefix = filter_by_prefix(df_google_Filter_out, prefix_mapping)

In [ ]:
df_filtered_prefix.count()

In [ ]:
df_filtered_prefix.show(100, truncate=False)

**Clean data with invalid ZIP code**

In [ ]:
# Dictionary mapping country names (lowercase) to expected ZIP code regex patterns.
zip_code_patterns = {
    "brunei": r"^\d{5}$",                         # Example pattern; adjust as needed.
    "india": r"^\d{6}$",
    "sweden": r"^\d{3}\s?\d{2}$",                  # Swedish postal codes: e.g., "123 45"
    "vanuatu": r"^\d{4}$",
    "nepal": r"^\d{5}$",
    "bangladesh": r"^\d{4}$",                      # Adjust if needed.
    "gabon": r"^\d{3}$",
    "kazakhstan": r"^\d{6}$",
    "israel": r"^\d{5,7}$",                        # Varies; adjust as needed.
    "romania": r"^\d{6}$",
    "somalia": r"^\d{5}$",
    "united states": r"^\d{5}(-\d{4})?$",          # 5-digit or ZIP+4.
    "el salvador": r"^\d{4}$",
    "saint lucia": r"^\d{5}$",
    "austria": r"^\d{4}$",
    "niger": r"^\d{5}$",
    "russia": r"^\d{6}$",
    "hong kong": r"^\d{5}$",                       # Hong Kong rarely uses postal codes.
    "suriname": r"^\d{5}$",
    "bulgaria": r"^\d{4}$",
    "qatar": r"^\d{4}$",
    "panama": r"^\d{4}$",
    "latvia": r"^\d{4}$",
    "mauritius": r"^\d{3}$",
    "guam": r"^\d{5}$",                           # Same as US.
    "venezuela": r"^\d{4}$",
    "japan": r"^\d{3}-\d{4}$",
    "serbia": r"^\d{5}$",
    "greece": r"^\d{5}$",
    "italy": r"^\d{5}$",
    "denmark": r"^\d{4}$",
    "comoros": r"^\d{3}$",
    "netherlands": r"^\d{4}\s?[A-Z]{2}$",         # e.g., "1234 AB"
    "china": r"^\d{6}$",
    "papua new guinea": r"^\d{4}$",
    "bahrain": r"^\d{3}$",
    "ireland": r"^[A-Z0-9]{3,4}\s?[A-Z0-9]{3}$",   # Irish Eircode (rough approximation)
    "tonga": r"^\d{3}$",
    "zambia": r"^\d{5}$",
    "fiji": r"^\d{3}$",
    "dominica": r"^\d{5}$",
    "faroe islands": r"^\d{3}$",
    "czechia": r"^\d{3}\s?\d{2}$",
    "liberia": r"^\d{5}$",
    "switzerland": r"^\d{4}$",
    "guinea": r"^\d{5}$",
    "republic of lithuania": r"^\d{5}$",
    "egypt": r"^\d{5}$",
    "senegal": r"^\d{5}$",
    "ukraine": r"^\d{5}$",
    "armenia": r"^\d{4}$",
    "philippines": r"^\d{4}$",
    "zimbabwe": r"^\d{4}$",
    "benin": r"^\d{4}$",
    "aruba": r"^\d{4}$",
    "sudan": r"^\d{5}$",
    "haiti": r"^\d{4}$",
    "bahamas": r"^\d{4}$",
    "republic of moldova": r"^\d{4}$",
    "saudi arabia": r"^\d{5}$",
    "chile": r"^\d{7}$",                         # Chilean postal codes are 7 digits.
    "iceland": r"^\d{3}$",
    "kyrgyzstan": r"^\d{6}$",
    "guyana": r"^\d{6}$",
    "south africa": r"^\d{4}$",
    "trinidad and tobago": r"^\d{4}$",
    "mexico": r"^\d{5}$",
    "united kingdom": r"^[A-Z]{1,2}\d[A-Z\d]?\s*\d[A-Z]{2}$",
    "uzbekistan": r"^\d{6}$",
    "sierra leone": r"^\d{5}$",
    "cuba": r"^\d{5}$",
    "tunisia": r"^\d{4}$",
    "slovakia": r"^\d{3}\s?\d{2}$",
    "georgia": r"^\d{5}$",
    "yemen": r"^\d{5}$",
    "dominican republic": r"^\d{5}$",
    "cambodia": r"^\d{5}$",
    "argentina": r"^\d{4}$",
    "ghana": r"^\d{5}$",
    "slovenia": r"^\d{4}$",
    "indonesia": r"^\d{5}$",
    "myanmar": r"^\d{5}$",
    "peru": r"^\d{5}$",
    "belgium": r"^\d{4}$",
    "tajikistan": r"^\d{6}$",
    "gambia": r"^\d{5}$",
    "belarus": r"^\d{6}$",
    "liechtenstein": r"^\d{4}$",
    "estonia": r"^\d{5}$",
    "iraq": r"^\d{5}$",
    "uruguay": r"^\d{5}$",
    "libya": r"^\d{5}$",
    "mauritania": r"^\d{5}$",
    "singapore": r"^\d{6}$",
    "colombia": r"^\d{6}$",
    "luxembourg": r"^\d{4}$",
    "paraguay": r"^\d{4}$",
    "barbados": r"^\d{5}$",
    "poland": r"^\d{2}-\d{3}$",                  # Polish postal code: e.g., "00-001"
    "spain": r"^\d{5}$",
    "burkina faso": r"^\d{5}$",
    "mali": r"^\d{5}$",
    "portugal": r"^\d{4}-\d{3}$",                # e.g., "1000-001"
    "algeria": r"^\d{5}$",
    "curaçao": r"^\d{4}$",
    "north macedonia": r"^\d{4}$",
    "samoa": r"^\d{3}$",
    "sint maarten": r"^\d{4}$",
    "thailand": r"^\d{5}$",
    "nigeria": r"^\d{6}$",
    "martinique": r"^\d{5}$",
    "costa rica": r"^\d{5}$",
    "namibia": r"^\d{5}$",
    "french polynesia": r"^\d{3}$",
    "germany": r"^\d{5}$",
    "croatia": r"^\d{5}$",
    "oman": r"^\d{5}$",
    "hungary": r"^\d{4}$",
    "united arab emirates": r"^\d{5}$",
    "kuwait": r"^\d{5}$",
    "canada": r"^[A-Z]\d[A-Z] \d[A-Z]\d$",
    "uganda": r"^\d{5}$",
    "brazil": r"^\d{5}-\d{3}$",
    "albania": r"^\d{4}$",
    "taiwan": r"^\d{3,5}$",
    "mozambique": r"^\d{5}$",
    "seychelles": r"^\d{3}$",
    "puerto rico": r"^\d{5}$",
    "malta": r"^\d{3,4}$",
    "u.s. virgin islands": r"^\d{5}$",
    "bosnia and herzegovina": r"^\d{5}$",
    "south korea": r"^\d{5}$",
    "rwanda": r"^\d{5}$",
    "central african republic": r"^\d{5}$",
    "andorra": r"^\d{3}$",
    "sri lanka": r"^\d{5}$",
    "ecuador": r"^\d{5}$",
    "morocco": r"^\d{5}$",
    "guadeloupe": r"^\d{5}$",
    "afghanistan": r"^\d{4}$",
    "azerbaijan": r"^\d{5}$",
    "guatemala": r"^\d{5}$",
    "botswana": r"^\d{5}$",
    "togo": r"^\d{5}$",
    "malaysia": r"^\d{5}$",
    "solomon islands": r"^\d{3}$",
    "norway": r"^\d{4}$",
    "burundi": r"^\d{5}$",
    "montenegro": r"^\d{5}$",
    "cameroon": r"^\d{5}$",
    "bolivia": r"^\d{5}$",
    "angola": r"^\d{5}$",
    "malawi": r"^\d{5}$",
    "honduras": r"^\d{5}$",
    "belize": r"^\d{5}$",
    "new zealand": r"^\d{4}$",
    "pakistan": r"^\d{5}$",
    "vietnam": r"^\d{5}$",
    "bhutan": r"^\d{5}$",
    "australia": r"^\d{4}$",
    "jamaica": r"^\d{5}$",
    "turkey": r"^\d{5}$",
    "åland": r"^\d{5}$",
    "nicaragua": r"^\d{5}$",
    "finland": r"^\d{5}$",
    "ethiopia": r"^\d{5}$",
    "madagascar": r"^\d{5}$",
    "laos": r"^\d{5}$",
    "lebanon": r"^\d{5}$",
    "maldives": r"^\d{5}$",
    "france": r"^\d{5}$",
    "saint vincent and the grenadines": r"^\d{5}$",
    "french guiana": r"^\d{5}$",
    "reunion": r"^\d{5}$",
    "lesotho": r"^\d{5}$",
    "american samoa": r"^\d{5}$",
    "kenya": r"^\d{5}$",
    "south sudan": r"^\d{5}$",
    "guinea-bissau": r"^\d{5}$",
    "isle of man": r"^\d{5}$",
    "greenland": r"^\d{5}$",
    "new caledonia": r"^\d{5}$",
    "jersey": r"^\d{5}$",
    "hashemite kingdom of jordan": r"^\d{5}$",
    "bermuda": r"^\d{5}$",
    "iran": r"^\d{5}$",
    "st kitts and nevis": r"^\d{5}$"
}

In [ ]:
from pyspark.sql.functions import lower, col

def filter_by_zip_code(df, zip_code_patterns):
    """
    Filters the given DataFrame based on country-specific ZIP code patterns.

    Parameters:
    df (DataFrame): The input DataFrame containing 'country_name' and 'zip_code' columns.
    zip_code_patterns (dict): Dictionary mapping country names to their respective ZIP code regex patterns.

    Returns:
    DataFrame: A filtered DataFrame containing only rows with valid ZIP codes.
    """
    zip_conditions = None

    for country, pattern in zip_code_patterns.items():
        cond = (lower(col("country_name")) == country) & col("zip_code").rlike(pattern)

        if zip_conditions is None:
            zip_conditions = cond
        else:
            zip_conditions = zip_conditions | cond

    if zip_conditions is not None:
        return df.filter(zip_conditions)
    return df  # Return original DataFrame if no conditions were created

In [ ]:
# Usage
df_filtered_zip = filter_by_zip_code(df_filtered_prefix, zip_code_patterns)

In [ ]:
# Display the results
df_filtered_zip.show(100, truncate=False)

In [ ]:
df_filtered_zip.count()

In [ ]:
from pyspark.sql.functions import col

# Update the phone_country_code column to have the same value as country_code
df_filtered_zip = df_filtered_zip.withColumn("phone_country_code", col("country_code"))

# Display the updated DataFrame
df_filtered_zip.show(500, truncate=False)

In [ ]:
df_final_google=df_filtered_zip.fillna('0')

In [ ]:
df_final_google.show(10000, truncate=False)

In [ ]:
from pyspark.sql.functions import regexp_replace

df_final_google = df_final_google.withColumn(
    "raw_address",
    regexp_replace("raw_address", r"\d+\+?\s*years in business\s*·?\s*", "")
)


In [ ]:
df_final_google.show(10000, truncate=False)

In [ ]:
from pyspark.sql.functions import regexp_replace

df_final_google_cleaned = df_final_google.withColumn(
    "raw_address",
    regexp_replace("raw_address", r" · .*", "")
)


In [ ]:
df_final_google_cleaned.show(10000, truncate=False)

**Load facebook dataset**

In [ ]:
spark = SparkSession.builder.getOrCreate()
file_path_facebook="/content/drive/MyDrive/datasets_interview/facebook_dataset.csv"
df_facebook=spark.read.csv(file_path_facebook, header=True, inferSchema=True)

In [ ]:
df_facebook.show(500)

In [ ]:
df_facebook.printSchema()

**Clean facebook dataset**

In [ ]:
# Update the DataFrame with the new country_code column
df_facebook = add_country_code(df_facebook, country_code_mapping)

# Show updated DataFrame
df_facebook.show(1000)

In [ ]:
# Select distinct country_name and country_code pairs
distinct_countries_facebook = df_facebook.select("country_name", "country_code").distinct()
n2=distinct_countries.count()
print(n2)

In [ ]:
# Show the distinct pairs of country names and country codes
distinct_countries_facebook.show(n2,truncate=False)

In [ ]:
from pyspark.sql.functions import col

# Filter out rows where the 'address' column is null
df_facebook_Filter_out = df_facebook.filter(col("address").isNotNull())

# Show the cleaned DataFrame
df_facebook_Filter_out.count()

In [ ]:
df_facebook_Filter_out.show(1000)

In [ ]:
# Filter out rows with null values in 'country_code', 'city', and 'country_name'
df_facebook_cleaned = df_facebook_Filter_out.filter(
    col("country_code").isNotNull() & col("city").isNotNull() & col("country_name").isNotNull()
)

# Show the cleaned DataFrame
df_facebook_cleaned.show(1000)

In [ ]:
from pyspark.sql.functions import lower, col


# Filter the DataFrame so that only rows with valid ZIP codes remain.
df_facebook_cleaned_zip = filter_by_zip_code(df_facebook_cleaned, zip_code_patterns)

#display the results.
df_facebook_cleaned_zip.show(100, truncate=False)

In [ ]:
from pyspark.sql.functions import lower, col, regexp_replace

# Apply the combined condition to filter the DataFrame
df_facebook_cleaned_prefix = filter_by_prefix(df_facebook_cleaned_zip, prefix_mapping)

# Show all rows from the filtered DataFrame
df_facebook_cleaned_prefix.show(100, truncate=False)

In [ ]:
# Update the phone_country_code column to have the same value as country_code
df_facebook_cleaned_prefix = df_facebook_cleaned_prefix.withColumn("phone_country_code", col("country_code"))

# Display the updated DataFrame
df_facebook_cleaned_prefix.show(500, truncate=False)

In [ ]:
from pyspark.sql.functions import when, col, regexp_extract

# Assuming df_final_google is your DataFrame
df_facebook_cleaned_region_code = df_facebook_cleaned_prefix.withColumn(
    "region_code",
    when(
        col("region_code").rlike(r"\d+"), "0"
    ).otherwise(col("region_code"))
)
df_facebook_cleaned_region_code.show(500)

In [ ]:
df_final_facebook = df_facebook_cleaned_region_code.fillna('0')
df_final_facebook.show(10000)

In [ ]:
df_final_facebook.count()

**Load website dataset**

In [ ]:
file_path_website="/content/drive/MyDrive/datasets_interview/website_dataset.csv"
df_website=spark.read.csv(file_path_google, header=True, inferSchema=True)

In [ ]:
df_website.count()

In [ ]:
df_website.printSchema()

In [ ]:
df_website.show(1000)

**Clean website Dataset**

In [ ]:
from pyspark.sql.functions import col

# Filter out rows where the 'address' column is null
df_website_Filter_out = df_website.filter(col("address").isNotNull())

# Show the cleaned DataFrame
df_website_Filter_out.count()

In [ ]:
df_website_Filter_out.show(1000)

In [ ]:
# Update the DataFrame with the new country_code column
df_website_Filter_out = add_country_code(df_website_Filter_out, country_code_mapping)

# Show updated DataFrame
df_website_Filter_out.show(1000)

In [ ]:
df_website_Filter_out.count()

In [ ]:
# Apply the combined condition to filter the DataFrame
df_website_Filter_out_prefix = filter_by_prefix(df_website_Filter_out, prefix_mapping)

# Show all rows from the filtered DataFrame
filtered_count = df_website_Filter_out_prefix.count()
print(filtered_count)

In [ ]:
df_website_Filter_out_prefix.show(1000, truncate=False)

In [ ]:
# Update the phone_country_code column to have the same value as country_code
df_website_Filter_out_prefix = df_website_Filter_out_prefix.withColumn("phone_country_code", col("country_code"))

# Display the updated DataFrame
df_website_Filter_out_prefix.show(1000, truncate=False)

In [ ]:
#delete from raw_address values that contains numbers+ years in business

from pyspark.sql.functions import regexp_replace

df_website_Filter_out_prefix = df_website_Filter_out_prefix.withColumn(
    "raw_address",
    regexp_replace("raw_address", r"\d+\+?\s*years in business\s*·?\s*", "")
)


In [ ]:
df_website_Filter_out_prefix.show(1000, truncate=False)

In [ ]:
print(df_website_Filter_out_prefix.count())

In [ ]:
# Filter the DataFrame so that only rows with valid ZIP codes remain.
df_website_Filter_out_zip = filter_by_zip_code(df_website_Filter_out_prefix, zip_code_patterns)

#display the results.
df_website_Filter_out_zip.show(1000, truncate=False)

In [ ]:
df_final_website = df_website_Filter_out_zip.fillna('0')
df_final_website.show(10000, truncate=False)

In [ ]:
from pyspark.sql.functions import regexp_replace

df_final_website_cleaned = df_final_website.withColumn(
    "raw_address",
    regexp_replace("raw_address", r" · .*", "")
)

In [ ]:
df_final_website_cleaned.show(10000, truncate=False)

**Create the final dataset**

In [ ]:
# Rename columns in Google dataset
df_final_google_cleaned = df_final_google_cleaned \
    .withColumnRenamed("address", "address_google") \
    .withColumnRenamed("category", "category_google") \
    .withColumnRenamed("phone", "phone_google") \
    .withColumnRenamed("raw_address", "raw_address_google")

# Rename columns in Facebook dataset
df_final_facebook = df_final_facebook \
    .withColumnRenamed("address", "address_facebook") \
    .withColumnRenamed("categories", "category_facebook") \
    .withColumnRenamed("phone", "phone_facebook") \
    .withColumnRenamed("email", "email_facebook") \
    .withColumnRenamed("link", "link_facebook")

# Rename columns in Website dataset
df_final_website_cleaned = df_final_website_cleaned \
    .withColumnRenamed("address", "address_website") \
    .withColumnRenamed("category", "category_website") \
    .withColumnRenamed("phone", "phone_website") \
    .withColumnRenamed("raw_address", "raw_address_website")\
    .withColumnRenamed("link", "link_website")


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import coalesce

# Initialize Spark Session
spark = SparkSession.builder.appName("DataJoin").getOrCreate()

# Perform the Join (Full Outer Join to capture all data)
df_joined = df_final_google_cleaned.join(df_final_facebook, "domain", "full_outer") \
                     .join(df_final_website_cleaned, "domain", "full_outer")

# Show result
df_joined.show(100)

In [ ]:
df_cleaned = df_joined.withColumn("final_address", coalesce("address_google", "address_facebook", "address_website")) \
                      .withColumn("final_category", coalesce("category_google", "category_facebook", "category_website")) \
                      .withColumn("final_phone", coalesce("phone_google", "phone_facebook", "phone_website"))

In [ ]:
df_final = df_cleaned.select("domain", "final_address", "final_category", "final_phone", "email_facebook", "link_facebook")
df_final.show(10000)

In [ ]:
df_final.count()

In [ ]:
# Assuming df_final is your final DataFrame
df_final = df_final.dropDuplicates()
df_final.show(10000)

In [ ]:
df_final.count()

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define the output path in your Google Drive
output_path = "/content/drive/MyDrive/final_dataset"

# Save the DataFrame as a CSV file
df_final.coalesce(1).write.csv(output_path, header=True, mode="overwrite")